In [5]:
import Bayesian_Inverse as BI 
import Solver3 as slice
import numpy as np

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from concurrent.futures import ProcessPoolExecutor

def run_single_chain(seed, beta_true, number_of_iter, burn_in, sigma, num_points):
    try:
        np.random.seed(seed)
        return BI.MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)
    except Exception as e:
        print(f"Error in run_single_chain with seed {seed}: {e}")
        raise


def run_multiple_chains(n_chains, beta_true, number_of_iter, burn_in, sigma, num_points):
    seeds = np.random.randint(0, 10000, size=n_chains)
    chains = []
    beta_mcmcs = []
    acceptance_histories = []
    acceptance_counts = []
    
    # Run chains in parallel using ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        # Submit all the chains concurrently
        futures = [
            executor.submit(run_single_chain, seed, beta_true, number_of_iter, burn_in, sigma, num_points)
            for seed in seeds
        ]
        # Collect results as they complete
        for future in futures:
            chain, beta_mcmc, acceptance_prob_history, acceptance_count = future.result()
            chains.append(chain)
            beta_mcmcs.append(beta_mcmc)
            acceptance_histories.append(acceptance_prob_history)
            acceptance_counts.append(acceptance_count)
    
    return chains, beta_mcmcs, acceptance_histories, acceptance_counts

if __name__ == '__main__':
    n_chains = 4
    number_of_iter = 10000
    burn_in = 5000
    sigma = 0.01 
    num_points = 100
    beta_true = np.array([0.5, 1/6])
    chains, beta_mcmcs, acceptance_counts, ddof_list = run_multiple_chains(
        n_chains, beta_true, number_of_iter, burn_in, sigma, num_points
    )
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmcs)
print("Acceptance Count:", acceptance_counts)



BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.